## LIBRERIAS y CONSTANTES

In [1]:
import numpy as np
import pandas as pd
import pickle
import torch.nn as nn
from torch.utils.data import DataLoader
import json
import plotly.express as px

#Implement training process
from model_trees_algebra import NeoRegression

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

class BaoTrainingException(Exception):
    pass

URL = "/media/data/ccarmona/memoria/dataset/"

IS CUDA AVAILABLE: True


## CLEAN DATA

In [2]:
def split_ds(all_data, val_rate, seed):
    """
    Used  to keep a balance of sets with respect to runtime of queries. 
    test_rate is a rate of the total,
    val_rate is a rate of the (total - test_rate)
    :param all_data: Pandas dataframe with data
    :param val_rate: Rate of the (total - test_rate)
    :param seed: For replication of results, this fixes the seed of split method. 
    :return: 
    """
    ranges = {}
    ranges['1_2'] = all_data[(all_data["time"] > 0)    & (all_data["time"] <= 2)]
    ranges['2_3'] = all_data[(all_data["time"] > 2)    & (all_data["time"] <= 3)]
    ranges['3_4'] = all_data[(all_data["time"] > 3)    & (all_data["time"] <= 4)]
    ranges['4_5'] = all_data[(all_data["time"] > 4)    & (all_data["time"] <= 5)]
    ranges['5_8'] = all_data[(all_data["time"] > 5)    & (all_data["time"] <= 8)]
    ranges['8_10'] = all_data[(all_data["time"] > 8)   & (all_data["time"] <= 10)]
    ranges['10_20'] =   all_data[(all_data["time"] > 10) & (all_data["time"] <= 20)]
    ranges['20_30'] =   all_data[(all_data["time"] > 20) & (all_data["time"] <= 30)]
    ranges['30_40'] =   all_data[(all_data["time"] > 30) & (all_data["time"] <= 40)]
    ranges['40_50'] =   all_data[(all_data["time"] > 40) & (all_data["time"] <= 50)]
    ranges['50_60'] =   all_data[(all_data["time"] > 50) & (all_data["time"] <= 60)]
    ranges['60_80'] =   all_data[(all_data["time"] > 60) & (all_data["time"] <= 80)]
    ranges['80_100'] =  all_data[(all_data["time"] > 80) & (all_data["time"] <= 100)]
    ranges['100_150'] = all_data[(all_data["time"] > 100) & (all_data["time"] <= 150)]
    ranges['150_200'] = all_data[(all_data["time"] > 150) & (all_data["time"] <= 200)]
    ranges['200_250'] = all_data[(all_data["time"] > 200) & (all_data["time"] <= 250)]
    ranges['250_last'] = all_data[(all_data["time"] > 250)]
    train_data = []
    val_data = []
    for rang in ranges.values():
        if rang.shape[0] >= 3:
            X_train, X_val = train_test_split(
                rang, test_size=val_rate, shuffle=True,random_state=seed)

            train_data.append(X_train)
            val_data.append(X_val)
    train_data_list = pd.concat(train_data)
    val_data_list = pd.concat(val_data)
    print("Shapes : Train: {} Val: {}".format(train_data_list.shape, val_data_list.shape))
    return train_data_list, val_data_list

data_train_val_raw = pd.read_csv(URL + "ds_trainval_pred_filtered.csv", delimiter="ᶶ", engine='python')
ds_test_raw = pd.read_csv(URL + "ds_test_pred_filtered.csv", delimiter="ᶶ", engine='python')

data_train_val_prev = data_train_val_raw[data_train_val_raw['time'] <=65]
ds_test_prev = ds_test_raw[ds_test_raw['time'] <=65]

print("Shape: train_val data" , data_train_val_prev.shape)
def clear_error_tuples(x):
    try:
        #if x == '["VAR_URI_LITERALᶲhttp://www.wikidata.org/prop/direct/P2529"]':
         #   print(x)
        #print(type(x))
        json.loads(x)
        return True
    except:
        print("Error in data ignored!", x)
        return False

#Split Dataset
ds_train_prev, ds_val_prev = split_ds(data_train_val_prev, 0.2,seed=None)
#Remove bad rows
ds_train  = ds_train_prev[ds_train_prev['trees'].apply(lambda x: clear_error_tuples(x))]
ds_val  = ds_val_prev[ds_val_prev['trees'].apply(lambda x: clear_error_tuples(x))]
ds_test  = ds_test_prev[ds_test_prev['trees'].apply(lambda x: clear_error_tuples(x))]


x = ds_test_prev['trees'][20]
print(x)
x = json.loads(x)
print(x)

Shape: train_val data (21703, 84)
Shapes : Train: (17357, 84) Val: (4346, 84)
Error in data ignored! [bgp "VAR_URI_VARᶲhttp://www.wikidata.org/prop/direct/P685"]
Error in data ignored! [datasetnames ?var1]
Error in data ignored! [reduced   ["VAR_VAR_VARᶲNONE"]]
Error in data ignored! [reduced   ["VAR_VAR_VARᶲNONE"]]
Error in data ignored! [reduced   ["VAR_VAR_VARᶲNONE"]]
Error in data ignored! [reduced   ["VAR_VAR_VARᶲNONE"]]
[ "JOINᶲhttp://www.wikidata.org/prop/direct/P31ᶲhttp://schema.org/description" ,   [ "VAR_URI_URIᶲhttp://www.wikidata.org/prop/direct/P31" ] ,   [ "VAR_URI_LITERALᶲhttp://schema.org/description" ] ] 
['JOINᶲhttp://www.wikidata.org/prop/direct/P31ᶲhttp://schema.org/description', ['VAR_URI_URIᶲhttp://www.wikidata.org/prop/direct/P31'], ['VAR_URI_LITERALᶲhttp://schema.org/description']]


In [18]:
data_train_val_prev = data_train_val_prev.reset_index(drop=True)
f = open("old_queries.txt", "a")
c = 0
for i in range(len(data_train_val_prev)):
    f.write(str(data_train_val_prev['query'][i])+"\n")
    c += 1
print(c)
f.close()

21703


In [19]:
ds_test_prev = ds_test_prev.reset_index(drop=True)
f2 = open("old_queries2.txt", "a")
c = 0
for i in range(len(ds_test_prev)):
    f2.write(str(data_train_val_prev['query'][i]) + "\n")
    c += 1
print(c)
f2.close()

2963


In [5]:
print("---------SHAPES-----------")
print("----------RAW-----------")
print(f'shape data_train_val_raw: {data_train_val_raw.shape}')
print(f'shape ds_test_raw: {ds_test_raw.shape}')
print("----------PREV----------")
print(f'shape ds_train_prev: {ds_train_prev.shape}')
print(f'shape ds_val_prev: {ds_val_prev.shape}')
print(f'shape ds_test_prev: {ds_test_prev.shape}')
print("----------CLEAN----------")
print(f'shape ds_train: {ds_train.shape}')
print(f'shape ds_val: {ds_val.shape}')
print(f'shape ds_test: {ds_test.shape}')
print("-----------------------")

---------SHAPES-----------
----------RAW-----------
shape data_train_val_raw: (25805, 84)
shape ds_test_raw: (3528, 84)
----------PREV----------
shape ds_train_prev: (17357, 84)
shape ds_val_prev: (4346, 84)
shape ds_test_prev: (2963, 84)
----------CLEAN----------
shape ds_train: (17351, 84)
shape ds_val: (4346, 84)
shape ds_test: (2963, 84)
-----------------------


In [6]:
data_train_val_raw['time'].describe()

count    25805.000000
mean        33.088488
std         55.198294
min          1.392783
25%          2.066000
50%          5.527993
75%         41.633546
max        307.183215
Name: time, dtype: float64

In [7]:
ds_test_raw['time'].describe()

count    3528.000000
mean       33.105443
std        55.097697
min         1.839000
25%         2.063000
50%         5.629112
75%        41.445001
max       305.150172
Name: time, dtype: float64

In [8]:
ds_train['time'].describe()

count    17351.000000
mean        12.972844
std         17.050058
min          1.392783
25%          2.045000
50%          2.290000
75%         18.915900
max         64.962858
Name: time, dtype: float64

In [9]:
ds_val_prev['time'].describe()

count    4346.000000
mean       13.028326
std        17.077475
min         1.829000
25%         2.047000
50%         2.308500
75%        19.255744
max        64.673409
Name: time, dtype: float64

In [10]:
ds_test['time'].describe()

count    2963.000000
mean       12.878044
std        16.911206
min         1.839000
25%         2.042000
50%         2.291000
75%        18.995694
max        64.915283
Name: time, dtype: float64

In [11]:
def getpredictions_info(x_val_tree, x_val_query, y_val):
    """
    Get statistics by a set of data. Need the previous trained model(availablre  form reg object).
    :param x_val_tree: Plan level features.
    :param x_val_query: Query level features.
    :param y_val: Real execution time
    :return: Dict with predictions and metrics (mae, rmse, mse)
    """
    Xt, Xq, Yv = reg.json_loads(x_val_tree, x_val_query.values, y_val)
    Xt = [reg.fix_tree(x) for x in Xt]
    Xt = reg.tree_transform.transform(Xt)

    pairs_val = list(zip(list(zip(Xt, Xq)), Yv))
    dataset_val = DataLoader(pairs_val, batch_size=64, num_workers=0, shuffle=False, collate_fn=reg.collate_with_card)
    results_val = reg.predict_best(dataset_val)
    y_pred_val, y_real_val = zip(*results_val)
    mseval = mean_squared_error(y_real_val, y_pred_val)
    maeval = mean_absolute_error(y_real_val, y_pred_val)
    rmseval = np.sqrt(mseval)
    return {"pred": y_pred_val, "real" : y_real_val, "mse": mseval, "mae": maeval, "rmse": rmseval, "history": reg.history}

def getmax(x):
    lista=  list(x.values())
    maximo = 0
    for el in lista:
        if (maximo < float(el)):
            maximo = float(el)
    return maximo

def pred2index_dict(x, pred_to_index, maxcardinality):
    """
    get histogram from cardinality features. the values is normalized using the max cardinality of predicate in dataset.
    :param x: Tree data from x row sample.
    :param pred_to_index: dict with predicates and their index.
    :param maxcardinality: Max cardiniality in the dataset.
    :return: dictionary with feature json_cardinality.
    """
    resp = {}
    x = json.loads(x)
    for el in x.keys():
        if el in pred_to_index:
            resp[pred_to_index[el]] = float(x[el])/maxcardinality
    return resp

def prepare_query_level_data(x_train_query, x_val_query, x_test_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    #Scale x_query data.
    xqtrain = x_train_query.drop(columns=['json_cardinality'])
    xqval   = x_val_query.drop(columns=['json_cardinality'])
    xqtest   = x_test_query.drop(columns=['json_cardinality'])

    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(xqtrain)
    x_val_scaled = scalerx.transform(xqval)
    x_test_scaled = scalerx.transform(xqtest)

    x_train_query =pd.concat([pd.DataFrame(x_train_scaled, index=xqtrain.index, columns=xqtrain.columns),x_train_query[['json_cardinality']]], axis=1)
    x_val_query =  pd.concat([pd.DataFrame(x_val_scaled,   index=xqval.index, columns=xqval.columns),x_val_query[['json_cardinality']]], axis=1)
    x_test_query =  pd.concat([pd.DataFrame(x_test_scaled,   index=xqtest.index, columns=xqtest.columns),x_test_query[['json_cardinality']]], axis=1)

    x_train_query['json_cardinality'] = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query['json_cardinality'] = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query['json_cardinality'] = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))

    return x_train_query, x_val_query, x_test_query


### TreeConv Neural Net

In [12]:
# Columns to use.
list_columns = ['bgp', 'distinct', 'extend', 'join', 'leftjoin', 'triple', 'union' ,
                'max_slice_start']
folds_execution = {}

print("Size Train: {}, Val {}".format(ds_train.shape[0], ds_val.shape[0]))


Size Train: 17351, Val 4346


In [13]:
# get query level data
x_train_query = ds_train[list_columns]
x_val_query   = ds_val[list_columns]

# get plan level data
x_train_tree = ds_train['trees'].values
x_val_tree = ds_val['trees'].values

y_train = ds_train['time'].values
y_val = ds_val['time'].values

x_test_tree = ds_test['trees'].values
y_test = ds_test['time'].values
x_test_query   = ds_test[list_columns]

In [14]:
print("---------SHAPES-----------")
print("----------CLEAN-----------")
print(f'shape ds_train: {ds_train.shape}')
print(f'shape ds_val  : {ds_val.shape}')
print(f'shape ds_test : {ds_test.shape}')
print("")
print("-----TRAIN AND VAL DATA-----")
print("----------x_query_data----------")
print(f'shape x_val_query  : {x_val_query.shape}')
print(f'shape x_train_query: {x_train_query.shape}')
print("----------x_plan_level_data----------")
print(f'shape x_val_tree  : {x_val_tree.shape}')
print(f'shape x_train_tree: {x_train_tree.shape}')
print("----------y_data------------")
print(f'shape y_val  : {y_val.shape}')
print(f'shape y_train: {y_train.shape}')
print("")
print("----------TEST DATA----------")
print(f'shape x_test_tree : {x_test_tree.shape}')
print(f'shape x_test_query: {x_test_query.shape}')
print(f'shape y_test      : {y_test.shape}')
print("-----------------------")


---------SHAPES-----------
----------CLEAN-----------
shape ds_train: (17351, 84)
shape ds_val  : (4346, 84)
shape ds_test : (2963, 84)

-----TRAIN AND VAL DATA-----
----------x_query_data----------
shape x_val_query  : (4346, 8)
shape x_train_query: (17351, 8)
----------x_plan_level_data----------
shape x_val_tree  : (4346,)
shape x_train_tree: (17351,)
----------y_data------------
shape y_val  : (4346,)
shape y_train: (17351,)

----------TEST DATA----------
shape x_test_tree : (2963,)
shape x_test_query: (2963, 8)
shape y_test      : (2963,)
-----------------------


In [15]:
maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
maxcardinality

KeyError: 'json_cardinality'

## NeoRegression
Esta en model_trees_algebra.py

In [ ]:
aec_dir = ''

verbose=True
reg = NeoRegression(
     aec={'train_aec': False, 'use_aec': True,'aec_file': '', 'aec_epochs': 200},
     epochs=400,
     maxcardinality=maxcardinality,
     in_channels_neo_net=512,
     tree_units=[512, 256, 128],
     tree_units_dense=[64, 32],
     early_stop_patience=10,
     early_stop_initial_patience=180,
     tree_activation_tree=nn.LeakyReLU,
     tree_activation_dense=nn.ReLU,
    optimizer={'optimizer': "Adam",'args':{"lr":0.00015}},
    figimage_size=(18,18),
    start_history_from_epoch=3
)


In [ ]:
from IPython.utils import io

In [ ]:
#Fit the transformer tree data
    ### Esto en el fondo mapea tanto  JOINS ,LEFT JOINS, los tipos de triple como VAR_URI_VAR, VAR_VAR_LITERAL ETC.. 
    ### ASI COMO TAMBIEN MAPEA LOS PREDICADOS a un indice
with io.capture_output() as captured:
    reg.fit_transform_tree_data_no_ds_rl(ds_train, ds_val, ds_test)
print("Trees tranformed!!!")


In [ ]:
with io.capture_output() as captured:
    x_train_query, x_val_query, x_test_query =  prepare_query_level_data(x_train_query, x_val_query, x_test_query)
print("END PREPARE QUERY LEVEL DATA")

In [ ]:
print("--------------dtype--------------------")
print("x_train_tree",x_train_tree.dtype)
print("x_train_query.values",x_train_query.values.dtype)
print("y_train",y_train.dtype)
print("x_val_tree",x_val_tree.dtype)
print("x_val_query.values",x_val_query.values.dtype)
print("y_val",y_val.dtype)
print("--------------TYPE--------------------")
print("x_train_tree",type(x_train_tree))
print("x_train_query.values",type(x_train_query.values))
print("y_train",type(y_train))
print("x_val_tree",type(x_val_tree))
print("x_val_query.values",type(x_val_query.values))
print("y_val",type(y_val))

In [ ]:
#Fit model
reg.fit(x_train_tree, x_train_query.values, y_train, x_val_tree, x_val_query.values, y_val)

In [ ]:
#Save best model
import torch
torch.save(reg.best_model.state_dict(), "./best_model.pt")
#Save stats in val set
file_to_store = open("./execution_model_stats.pickle", "wb")
pickle.dump(getpredictions_info(x_val_tree, x_val_query, y_val), file_to_store)
file_to_store.close()

In [ ]:
val_stats = getpredictions_info(x_val_tree, x_val_query, y_val)

In [ ]:
val_stats

In [ ]:
ds_val['y_pred'] = val_stats['pred']

In [ ]:
test_stats = getpredictions_info(x_test_tree, x_test_query, y_test)

In [ ]:
ds_test['y_pred'] = test_stats['pred']
ds_test['y_realcheck'] = test_stats['real']

In [ ]:
def tag_points(x):
    """Add quality tags of predictions. Used to plot with plotly"""
    difference = x['time'] - x['y_pred'][0]
    abs_diff = np.abs(difference)
    x['y_pred'] = x['y_pred'][0]
    x['query2'] = x['query'].replace(" . ", ' . <br>').replace(" FILTER", '<br> FILTER').replace(" { ", ' { <br>').replace(" } ", ' <br> }').replace(" ; ", ' ; <br>') 
    p20 = x['time'] * 0.2
    p40 = x['time'] * 0.4
    if abs_diff < p20:
        x['color'] = "good prediction"
    elif abs_diff < p40:
        x['color'] = "aceptable prediction"
    else:
        x['color'] = "bad prediction"
    return x

In [ ]:
other = ds_test.apply(lambda x: tag_points(x), axis=1)

In [ ]:
other.to_pickle("./predictions_test.pickle")

In [ ]:
otherval = ds_val.apply(lambda x: tag_points(x), axis=1)

In [ ]:
otherval.to_pickle("./predictions_val.pickle")

In [ ]:
fig = px.scatter(otherval[['id','query','query2','time','y_pred','color']], x="y_pred", y="time", color="color", hover_data=['id', 'query2'])
fig.update_layout(height=800, width=1000, title_text="Predictions on Val Set")
fig.show()

In [ ]:
fig = px.scatter(other[['id','query','query2','time','y_pred','color']], x="y_pred", y="time", color="color", hover_data=['id', 'query2'])
fig.update_layout(height=800, width=1000, title_text="Predictions on Test Set")
fig.show()


In [ ]:
def MetricTotalAccuraccy(ds_final, validation=True):
    copy_ds_final = ds_final.copy()
    copy_ds_final = copy_ds_final.reset_index(drop=True)
    bad_pred = []
    accep_pred = []
    good_pred = []
    try:
        if validation:
            for i in range(len(copy_ds_final)):
               # print(i)
                if copy_ds_final['color'][i] == 'bad prediction':
                    bad_pred.append((copy_ds_final['time'][i],copy_ds_final['y_pred'][i]))
                if copy_ds_final['color'][i] == 'aceptable prediction':
                    accep_pred.append((copy_ds_final['time'][i],copy_ds_final['y_pred'][i]))
                if copy_ds_final['color'][i] == 'good prediction':
                    good_pred.append((copy_ds_final['time'][i],copy_ds_final['y_pred'][i]))
        else:
            for i in range(len(copy_ds_final)):
               # print(i)
                if copy_ds_final['color'][i] == 'bad prediction':
                    bad_pred.append((copy_ds_final['time'][i],copy_ds_final['y_realcheck'][i]))
                if copy_ds_final['color'][i] == 'aceptable prediction':
                    accep_pred.append((copy_ds_final['time'][i],copy_ds_final['y_realcheck'][i]))
                if copy_ds_final['color'][i] == 'good prediction':
                    good_pred.append((copy_ds_final['time'][i],copy_ds_final['y_realcheck'][i]))
    except:
        print("Its not ds_final")
        return 0,0,0,0,0,0,0
    b = len(bad_pred)
    a = len(accep_pred)
    g = len(good_pred)
    tot = b+a+g
    
    bp = (b/tot)*100
    ap = (a/tot)*100
    gp = (g/tot)*100
    
    return tot,b,a,g,bp,ap,gp



In [ ]:
tot_val,b_val,a_val,g_val,bp_val,ap_val,gp_val = MetricTotalAccuraccy(otherval)
print(f"Total predictions: {tot_val}")
print(f"Bad predictions: {b_val}, percentage {bp_val}%")
print(f"Acceptable predictions: {a_val}, percentage {ap_val}%")
print(f"Good predictions: {g_val}, percentage {gp_val}%")

print(f"Accuraccy: {100*(a_val+g_val)/tot_val}%")

In [ ]:
tot_val,b_val,a_val,g_val,bp_val,ap_val,gp_val = MetricTotalAccuraccy(other)
print(f"Total predictions: {tot_val}")
print(f"Bad predictions: {b_val}, percentage {bp_val}%")
print(f"Acceptable predictions: {a_val}, percentage {ap_val}%")
print(f"Good predictions: {g_val}, percentage {gp_val}%")

print(f"Accuraccy: {100*(a_val+g_val)/tot_val}%")